In [2]:
"""
高德地理编码脚本 - Jupyter Notebook版本
功能:
1. 使用高德API对struc_addr_clean进行地理编码
2. 提取经纬度信息
3. 保存完整响应和坐标
"""

import pandas as pd
import requests
import time
import json
from pathlib import Path
from tqdm import tqdm


def geocode_with_amap(input_file, api_key, output_file=None, delay=0.2, save_interval=10):
    """
    使用高德API进行地理编码
    
    Args:
        input_file: 输入xlsx文件路径
        api_key: 高德API密钥
        output_file: 输出xlsx文件路径(可选)
        delay: 每次请求之间的延迟(秒),默认0.2秒(避免超过QPS限制)
        save_interval: 保存历史副本的间隔(条数),默认10条
    
    Returns:
        df_geocoded: 地理编码后的DataFrame
    """
    
    # 读取数据
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    if 'struc_addr_clean' not in df.columns:
        raise ValueError("文件中缺少 'struc_addr_clean' 列")
    
    print(f"\n数据行数: {len(df)}")
    
    df_geocoded = df.copy()
    
    # 初始化新列
    df_geocoded['response'] = None
    df_geocoded['lng'] = None
    df_geocoded['lat'] = None
    df_geocoded['geocode_status'] = None
    
    # 设置输出文件路径
    if output_file is None:
        input_path = Path(input_file)
        output_file = input_path.parent / f"{input_path.stem}_geocoded.xlsx"
    
    output_path = Path(output_file)
    
    # 创建历史副本目录
    backup_dir = output_path.parent / f"{output_path.stem}_backups"
    backup_dir.mkdir(exist_ok=True, parents=True)
    
    print(f"历史副本将保存到: {backup_dir}")
    
    # 高德地理编码API
    base_url = "https://restapi.amap.com/v3/geocode/geo"
    
    # 统计信息
    success_count = 0
    fail_count = 0
    processed_count = 0
    
    print("\n开始地理编码...")
    print("="*60)
    
    # 使用tqdm显示进度条
    for idx, row in tqdm(df_geocoded.iterrows(), total=len(df_geocoded), desc="地理编码进度"):
        address = row['struc_addr_clean']
        
        # 跳过空地址
        if pd.isna(address) or address == '':
            df_geocoded.at[idx, 'geocode_status'] = 'empty_address'
            fail_count += 1
            processed_count += 1
            continue
        
        try:
            # 构建请求URL
            params = {
                'address': address,
                'key': api_key,
                'output': 'JSON'
            }
            
            # 发送请求
            response = requests.get(base_url, params=params, timeout=10)
            
            # 检查HTTP状态
            if response.status_code != 200:
                df_geocoded.at[idx, 'geocode_status'] = f'http_error_{response.status_code}'
                fail_count += 1
                processed_count += 1
                time.sleep(delay)
                continue
            
            # 解析响应
            result = response.json()
            
            # 保存完整响应(转为字符串)
            df_geocoded.at[idx, 'response'] = json.dumps(result, ensure_ascii=False)
            
            # 检查API响应状态
            if result.get('status') == '1' and result.get('count', '0') != '0':
                # 成功响应,提取第一个POI的经纬度
                geocodes = result.get('geocodes', [])
                
                if len(geocodes) > 0:
                    first_geocode = geocodes[0]
                    location = first_geocode.get('location', '')
                    
                    if location:
                        # location格式为 "经度,纬度"
                        lng_lat = location.split(',')
                        if len(lng_lat) == 2:
                            df_geocoded.at[idx, 'lng'] = float(lng_lat[0])
                            df_geocoded.at[idx, 'lat'] = float(lng_lat[1])
                            df_geocoded.at[idx, 'geocode_status'] = 'success'
                            success_count += 1
                        else:
                            df_geocoded.at[idx, 'geocode_status'] = 'invalid_location_format'
                            fail_count += 1
                    else:
                        df_geocoded.at[idx, 'geocode_status'] = 'no_location'
                        fail_count += 1
                else:
                    df_geocoded.at[idx, 'geocode_status'] = 'no_geocodes'
                    fail_count += 1
            else:
                # API返回失败
                info = result.get('info', 'unknown')
                infocode = result.get('infocode', 'unknown')
                df_geocoded.at[idx, 'geocode_status'] = f'api_error_{infocode}_{info}'
                fail_count += 1
            
        except requests.exceptions.Timeout:
            df_geocoded.at[idx, 'geocode_status'] = 'timeout'
            fail_count += 1
        except requests.exceptions.RequestException as e:
            df_geocoded.at[idx, 'geocode_status'] = f'request_error_{str(e)[:50]}'
            fail_count += 1
        except Exception as e:
            df_geocoded.at[idx, 'geocode_status'] = f'error_{str(e)[:50]}'
            fail_count += 1
        
        processed_count += 1
        
        # 每处理指定数量条记录,保存一个历史副本
        if processed_count % save_interval == 0:
            backup_file = backup_dir / f"backup_{processed_count:05d}.xlsx"
            df_geocoded.to_excel(backup_file, index=False)
            tqdm.write(f"  💾 已保存历史副本: {backup_file.name} (已处理 {processed_count}/{len(df_geocoded)} 条)")
        
        # 延迟避免超过QPS限制
        time.sleep(delay)
    
    print("\n" + "="*60)
    print("地理编码完成")
    print("="*60)
    
    # 统计结果
    status_counts = df_geocoded['geocode_status'].value_counts()
    
    print(f"\n地理编码统计:")
    print(f"{'状态':<30} {'数量':<10} {'比例'}")
    print("-" * 60)
    print(f"{'成功':<30} {success_count:<10} ({success_count/len(df_geocoded)*100:.1f}%)")
    print(f"{'失败':<30} {fail_count:<10} ({fail_count/len(df_geocoded)*100:.1f}%)")
    
    print(f"\n详细状态分布:")
    for status, count in status_counts.items():
        percentage = count / len(df_geocoded) * 100
        print(f"  {status:<28} {count:<10} ({percentage:.1f}%)")
    
    # 显示失败样例
    failed_mask = df_geocoded['geocode_status'] != 'success'
    failed_count = failed_mask.sum()
    
    if failed_count > 0:
        print(f"\n失败样例(前10条):")
        failed_samples = df_geocoded[failed_mask].head(10)
        for idx, row in failed_samples.iterrows():
            print(f"\n  行 {idx+2}:")
            print(f"    地址: {row['struc_addr_clean']}")
            print(f"    状态: {row['geocode_status']}")
    
    # 保存最终结果
    print(f"\n\n保存最终结果到: {output_file}")
    df_geocoded.to_excel(output_file, index=False)
    
    # 同时保存一个最终副本到backup目录
    final_backup = backup_dir / f"backup_final_{len(df_geocoded):05d}.xlsx"
    df_geocoded.to_excel(final_backup, index=False)
    print(f"保存最终副本到: {final_backup}")
    
    # 生成总结
    print("\n" + "="*60)
    print("地理编码摘要")
    print("="*60)
    print(f"总记录数: {len(df_geocoded)}")
    print(f"成功编码: {success_count} 条 ({success_count/len(df_geocoded)*100:.1f}%)")
    print(f"失败编码: {fail_count} 条 ({fail_count/len(df_geocoded)*100:.1f}%)")
    print(f"\n历史副本:")
    print(f"  保存间隔: 每 {save_interval} 条")
    print(f"  副本总数: {len(list(backup_dir.glob('*.xlsx')))} 个")
    print(f"  副本目录: {backup_dir}")
    print(f"\n新增列:")
    print(f"  - response: 完整API响应(JSON字符串)")
    print(f"  - lng: 经度")
    print(f"  - lat: 纬度")
    print(f"  - geocode_status: 地理编码状态")
    print("="*60)
    
    return df_geocoded


def analyze_geocode_results(df_geocoded):
    """
    分析地理编码结果
    """
    print("\n地理编码结果分析")
    print("="*60)
    
    # 成功率
    success_count = (df_geocoded['geocode_status'] == 'success').sum()
    total_count = len(df_geocoded)
    success_rate = success_count / total_count * 100
    
    print(f"成功率: {success_rate:.2f}%")
    print(f"成功: {success_count} / {total_count}")
    
    # 坐标范围
    valid_coords = df_geocoded[df_geocoded['geocode_status'] == 'success']
    
    if len(valid_coords) > 0:
        print(f"\n坐标范围:")
        print(f"  经度: {valid_coords['lng'].min():.6f} ~ {valid_coords['lng'].max():.6f}")
        print(f"  纬度: {valid_coords['lat'].min():.6f} ~ {valid_coords['lat'].max():.6f}")
    
    # 失败原因统计
    failed = df_geocoded[df_geocoded['geocode_status'] != 'success']
    if len(failed) > 0:
        print(f"\n失败原因分布:")
        fail_reasons = failed['geocode_status'].value_counts()
        for reason, count in fail_reasons.head(10).items():
            print(f"  {reason}: {count} 条")
    
    print("="*60)
    
    
df_geocoded = geocode_with_amap('output/C-uie清洗待整理&人工审核_species_processed.xlsx', API_KEY)
analyze_geocode_results(df_geocoded)

读取文件: output/C-uie清洗待整理&人工审核_species_processed.xlsx

数据行数: 4334
历史副本将保存到: output\C-uie清洗待整理&人工审核_species_processed_geocoded_backups

开始地理编码...


地理编码进度:   0%|          | 10/4334 [00:04<1:09:01,  1.04it/s]   

  💾 已保存历史副本: backup_00010.xlsx (已处理 10/4334 条)


地理编码进度:   0%|          | 20/4334 [00:09<1:07:02,  1.07it/s]    

  💾 已保存历史副本: backup_00020.xlsx (已处理 20/4334 条)


...

  💾 已保存历史副本: backup_04320.xlsx (已处理 4320/4334 条)


地理编码进度: 100%|█████████▉| 4329/4334 [36:49<00:01,  3.37it/s]      

  💾 已保存历史副本: backup_04330.xlsx (已处理 4330/4334 条)


地理编码进度: 100%|██████████| 4334/4334 [36:50<00:00,  1.96it/s]



地理编码完成

地理编码统计:
状态                             数量         比例
------------------------------------------------------------
成功                             3776       (87.1%)
失败                             558        (12.9%)

详细状态分布:
  success                      3776       (87.1%)
  api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT 556        (12.8%)
  api_error_30001_ENGINE_RESPONSE_DATA_ERROR 2          (0.0%)

失败样例(前10条):

  行 10:
    地址: 江苏省-南京市-玄武区
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 17:
    地址: 云南省-昆明市-嵩明县
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 21:
    地址: 湖南省-湘西土家族苗族自治州-花垣县
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 31:
    地址: 广西壮族自治区-百色市-右江区
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 38:
    地址: 广东省-中山市-中山市
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 48:
    地址: 广西壮族自治区-河池市-东兰县
    状态: api_error_10021_CUQPS_HAS_EXCEEDED_THE_LIMIT

  行 58:
    地址: 广西壮族自治区-南宁市-兴宁区
    状态: api_error_10021_CUQPS_HAS_EX

In [4]:
"""
高德地理编码重试脚本 - Jupyter Notebook版本
功能:
1. 读取包含失败记录的文件
2. 重新对失败记录进行地理编码
3. 循环重试直到没有异常响应
4. 每轮(epoch)保存一次结果
"""

import pandas as pd
import requests
import time
import json
from pathlib import Path
from tqdm import tqdm


def retry_failed_geocoding(input_file, api_key, output_file=None, delay=0.2, max_epochs=10):
    """
    重试失败的地理编码记录
    
    Args:
        input_file: 输入xlsx文件路径(包含失败记录的文件)
        api_key: 高德API密钥
        output_file: 输出xlsx文件路径(可选)
        delay: 每次请求之间的延迟(秒),默认0.2秒
        max_epochs: 最大重试轮数,默认10轮
    
    Returns:
        df_final: 最终结果DataFrame
    """
    
    # 读取数据
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    
    # 验证列名
    required_cols = ['struc_addr_clean', 'response', 'lng', 'lat', 'geocode_status']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"文件中缺少必需的列: {missing_cols}")
    
    print(f"\n原始数据行数: {len(df)}")
    
    # 设置输出文件路径
    if output_file is None:
        input_path = Path(input_file)
        output_file = input_path.parent / f"{input_path.stem}_retry_final.xlsx"
    
    output_path = Path(output_file)
    
    # 创建epoch结果目录
    epoch_dir = output_path.parent / f"{output_path.stem}_epochs"
    epoch_dir.mkdir(exist_ok=True, parents=True)
    
    print(f"每轮结果将保存到: {epoch_dir}")
    
    # 高德地理编码API
    base_url = "https://restapi.amap.com/v3/geocode/geo"
    
    # 开始重试循环
    df_current = df.copy()
    
    for epoch in range(1, max_epochs + 1):
        print("\n" + "="*60)
        print(f"第 {epoch} 轮重试")
        print("="*60)
        
        # 识别需要重试的记录
        failed_records = identify_failed_records(df_current)
        
        if len(failed_records) == 0:
            print("✓ 没有需要重试的记录,全部成功!")
            break
        
        print(f"需要重试的记录数: {len(failed_records)}")
        
        # 显示失败原因统计
        print("\n失败原因分布:")
        for reason, count in failed_records['failure_reason'].value_counts().head(10).items():
            print(f"  {reason}: {count} 条")
        
        # 重试失败的记录
        print(f"\n开始重试 {len(failed_records)} 条记录...")
        
        success_count = 0
        still_failed_count = 0
        
        for idx in tqdm(failed_records.index, desc=f"Epoch {epoch} 进度"):
            address = df_current.at[idx, 'struc_addr_clean']
            
            # 跳过空地址
            if pd.isna(address) or address == '':
                continue
            
            try:
                # 构建请求URL
                params = {
                    'address': address,
                    'key': api_key,
                    'output': 'JSON'
                }
                
                # 发送请求
                response = requests.get(base_url, params=params, timeout=10)
                
                # 检查HTTP状态
                if response.status_code != 200:
                    df_current.at[idx, 'geocode_status'] = f'http_error_{response.status_code}'
                    still_failed_count += 1
                    time.sleep(delay)
                    continue
                
                # 解析响应
                result = response.json()
                
                # 保存完整响应(转为字符串)
                df_current.at[idx, 'response'] = json.dumps(result, ensure_ascii=False)
                
                # 检查API响应状态
                if result.get('status') == '1' and result.get('count', '0') != '0':
                    # 成功响应,提取第一个POI的经纬度
                    geocodes = result.get('geocodes', [])
                    
                    if len(geocodes) > 0:
                        first_geocode = geocodes[0]
                        location = first_geocode.get('location', '')
                        
                        if location:
                            # location格式为 "经度,纬度"
                            lng_lat = location.split(',')
                            if len(lng_lat) == 2:
                                df_current.at[idx, 'lng'] = float(lng_lat[0])
                                df_current.at[idx, 'lat'] = float(lng_lat[1])
                                df_current.at[idx, 'geocode_status'] = 'success'
                                success_count += 1
                            else:
                                df_current.at[idx, 'geocode_status'] = 'invalid_location_format'
                                still_failed_count += 1
                        else:
                            df_current.at[idx, 'geocode_status'] = 'no_location'
                            still_failed_count += 1
                    else:
                        df_current.at[idx, 'geocode_status'] = 'no_geocodes'
                        still_failed_count += 1
                else:
                    # API返回失败
                    info = result.get('info', 'unknown')
                    infocode = result.get('infocode', 'unknown')
                    df_current.at[idx, 'geocode_status'] = f'api_error_{infocode}_{info}'
                    still_failed_count += 1
                
            except requests.exceptions.Timeout:
                df_current.at[idx, 'geocode_status'] = 'timeout'
                still_failed_count += 1
            except requests.exceptions.RequestException as e:
                df_current.at[idx, 'geocode_status'] = f'request_error_{str(e)[:50]}'
                still_failed_count += 1
            except Exception as e:
                df_current.at[idx, 'geocode_status'] = f'error_{str(e)[:50]}'
                still_failed_count += 1
            
            # 延迟避免超过QPS限制
            time.sleep(delay)
        
        # 保存本轮结果
        epoch_file = epoch_dir / f"epoch_{epoch:02d}.xlsx"
        df_current.to_excel(epoch_file, index=False)
        
        print(f"\n第 {epoch} 轮结果:")
        print(f"  本轮成功: {success_count} 条")
        print(f"  仍然失败: {still_failed_count} 条")
        print(f"  已保存到: {epoch_file.name}")
        
        # 统计当前状态
        current_failed = identify_failed_records(df_current)
        total_success = (df_current['geocode_status'] == 'success').sum()
        total_failed = len(current_failed)
        
        print(f"\n累计统计:")
        print(f"  总成功: {total_success} 条 ({total_success/len(df_current)*100:.1f}%)")
        print(f"  总失败: {total_failed} 条 ({total_failed/len(df_current)*100:.1f}%)")
        
        # 如果没有改进,停止重试
        if success_count == 0:
            print(f"\n本轮没有任何记录成功,停止重试")
            break
    
    # 保存最终结果
    print("\n" + "="*60)
    print("保存最终结果")
    print("="*60)
    
    df_current.to_excel(output_file, index=False)
    print(f"最终结果已保存到: {output_file}")
    
    # 生成最终报告
    print("\n" + "="*60)
    print("最终统计报告")
    print("="*60)
    
    final_success = (df_current['geocode_status'] == 'success').sum()
    final_failed = len(identify_failed_records(df_current))
    
    print(f"总记录数: {len(df_current)}")
    print(f"成功编码: {final_success} 条 ({final_success/len(df_current)*100:.2f}%)")
    print(f"失败编码: {final_failed} 条 ({final_failed/len(df_current)*100:.2f}%)")
    print(f"\n重试轮数: {epoch} 轮")
    print(f"Epoch结果: {epoch_dir}")
    
    if final_failed > 0:
        print(f"\n仍然失败的记录:")
        failed_final = identify_failed_records(df_current)
        for reason, count in failed_final['failure_reason'].value_counts().head(10).items():
            print(f"  {reason}: {count} 条")
    
    print("="*60)
    
    return df_current


def identify_failed_records(df):
    """
    识别失败的记录
    
    包括:
    1. geocode_status不为'success'
    2. response包含错误信息(如QPS超限)
    3. 经纬度为空
    """
    
    failed_mask = pd.Series([False] * len(df), index=df.index)
    failure_reasons = pd.Series([''] * len(df), index=df.index)
    
    for idx, row in df.iterrows():
        # 检查geocode_status
        if row['geocode_status'] != 'success':
            failed_mask[idx] = True
            failure_reasons[idx] = row['geocode_status'] if pd.notna(row['geocode_status']) else 'unknown'
            continue
        
        # 检查经纬度
        if pd.isna(row['lng']) or pd.isna(row['lat']):
            failed_mask[idx] = True
            failure_reasons[idx] = 'missing_coordinates'
            continue
        
        # 检查response中的错误
        if pd.notna(row['response']):
            try:
                resp = json.loads(row['response'])
                
                # 检查status
                if resp.get('status') == '0':
                    failed_mask[idx] = True
                    infocode = resp.get('infocode', 'unknown')
                    info = resp.get('info', 'unknown')
                    failure_reasons[idx] = f'api_error_{infocode}_{info}'
                    continue
                
                # 检查count
                if resp.get('count', '0') == '0':
                    failed_mask[idx] = True
                    failure_reasons[idx] = 'zero_results'
                    continue
                    
            except json.JSONDecodeError:
                failed_mask[idx] = True
                failure_reasons[idx] = 'invalid_json_response'
                continue
    
    failed_df = df[failed_mask].copy()
    failed_df['failure_reason'] = failure_reasons[failed_mask]
    
    return failed_df


def analyze_final_results(df):
    """
    分析最终结果
    """
    print("\n最终结果分析")
    print("="*60)
    
    # 成功率
    success_count = (df['geocode_status'] == 'success').sum()
    total_count = len(df)
    success_rate = success_count / total_count * 100
    
    print(f"成功率: {success_rate:.2f}%")
    print(f"成功: {success_count} / {total_count}")
    
    # 坐标范围
    valid_coords = df[df['geocode_status'] == 'success']
    
    if len(valid_coords) > 0:
        print(f"\n坐标范围:")
        print(f"  经度: {valid_coords['lng'].min():.6f} ~ {valid_coords['lng'].max():.6f}")
        print(f"  纬度: {valid_coords['lat'].min():.6f} ~ {valid_coords['lat'].max():.6f}")
    
    # 失败记录
    failed = identify_failed_records(df)
    if len(failed) > 0:
        print(f"\n失败记录 ({len(failed)} 条):")
        for reason, count in failed['failure_reason'].value_counts().head(10).items():
            print(f"  {reason}: {count} 条")
        
        print(f"\n失败样例(前5条):")
        for idx, row in failed.head(5).iterrows():
            print(f"\n  行 {idx+2}:")
            print(f"    地址: {row['struc_addr_clean']}")
            print(f"    原因: {row['failure_reason']}")
    
    print("="*60)



df_final = retry_failed_geocoding(
    'output/C-uie清洗待整理&人工审核_species_processed_geocoded_backups/backup_final_04334_.xlsx', 
    API_KEY,
    delay=0.3,           
    max_epochs=1,       
)
analyze_final_results(df_final)


读取文件: output/C-uie清洗待整理&人工审核_species_processed_geocoded_backups/backup_final_04334_.xlsx

原始数据行数: 4334
每轮结果将保存到: output\C-uie清洗待整理&人工审核_species_processed_geocoded_backups\backup_final_04334__retry_final_epochs

第 1 轮重试
需要重试的记录数: 2

失败原因分布:
  api_error_30001_ENGINE_RESPONSE_DATA_ERROR: 2 条

开始重试 2 条记录...


Epoch 1 进度: 100%|██████████| 2/2 [00:00<00:00,  2.72it/s]



第 1 轮结果:
  本轮成功: 2 条
  仍然失败: 0 条
  已保存到: epoch_01.xlsx

累计统计:
  总成功: 4334 条 (100.0%)
  总失败: 0 条 (0.0%)

保存最终结果
最终结果已保存到: output\C-uie清洗待整理&人工审核_species_processed_geocoded_backups\backup_final_04334__retry_final.xlsx

最终统计报告
总记录数: 4334
成功编码: 4334 条 (100.00%)
失败编码: 0 条 (0.00%)

重试轮数: 1 轮
Epoch结果: output\C-uie清洗待整理&人工审核_species_processed_geocoded_backups\backup_final_04334__retry_final_epochs

最终结果分析
成功率: 100.00%
成功: 4334 / 4334

坐标范围:
  经度: 75.862839 ~ 132.037659
  纬度: 16.831608 ~ 51.725924
